<a href="https://colab.research.google.com/github/lvlanson/data_mining/blob/main/Praktikum_2_Aufgabe_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# - Feature Selection: Select a subset of input features from the dataset.
#   - Unsupervised: Do not use the target variable (e.g. remove redundant variables).
#       - Correlation
#   - Supervised: Use the target variable (e.g. remove irrelevant variables).
#       - Wrapper: Search for well-performing subsets of features.
#           - RFE
#       - Filter: Select subsets of features based on their relationship with the target.
#           - Statistical Methods
#           - Feature Importance Methods
#   - Intrinsic: Algorithms that perform automatic feature selection during training.
#       - Decision Trees
# - Dimensionality Reduction: Project input data into a lower-dimensional feature space.

# Quelle: https://machinelearningmastery.com/feature-selection-with-real-and-categorical-data/

In [2]:
def filter_selection(X, selector):
  return X[X.columns[selector.get_support(indices=True)]]

In [3]:
import pandas as pd

url    = "https://raw.githubusercontent.com/lvlanson/data_mining/main/Praktikum_2/Bodyfat.csv"
data_frame = pd.read_csv(url, sep=";", error_bad_lines=False)

In [4]:
data_frame.describe()

,Dichte,Koerperfettanteil,Alter,Gewicht_lbs,Groesse_inch,Nackenumfang,Brustumfang,Bauchumfang,Hueftumfang,Obschenkelumfang,Knieumfang,Fussgelenkumfang,Oberarmumfang,Unterarmumfang,Handgelenkumfang
count,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000
mean,1.055574,19.150794,44.884921,178.924405,70.148810,37.992063,100.824206,92.555952,99.904762,59.405952,38.590476,23.102381,32.273413,28.663889,18.229762
std,0.019031,8.368740,12.602040,29.389160,3.662856,2.430913,8.430476,10.783077,7.164058,5.249952,2.411805,1.694893,3.021274,2.020691,0.933585
min,0.995000,0.000000,22.000000,118.500000,29.500000,31.100000,79.300000,69.400000,85.000000,47.200000,33.000000,19.100000,24.800000,21.000000,15.800000
25%,1.041400,12.475000,35.750000,159.000000,68.250000,36.400000,94.350000,84.575000,95.500000,56.000000,36.975000,22.000000,30.200000,27.300000,17.600000
50%,1.054900,19.200000,43.000000,176.500000,70.000000,38.000000,99.650000,90.950000,99.300000,59.000000,38.500000,22.800000,32.050000,28.700000,18.300000
75%,1.070400,25.300000,54.000000,197.000000,72.250000,39.425000,105.375000,99.325000,103.525000,62.350000,39.925000,24.000000,34.325000,30.000000,18.800000
max,1.108900,47.500000,81.000000,363.150000,77.750000,51.200000,136.200000,148.100000,147.700000,87.300000,49.100000,33.900000,45.000000,34.900000,21.400000


In [5]:
# a) Forward Feature Selection
# X - Feature Set
# y - Label was es vorherzusagen gilt

X = data_frame.drop("Nackenumfang", axis=1)
y = data_frame["Nackenumfang"]

In [6]:
# Feature Selection

# coefficient: gibt an wie weit im Schnitt sich der Wert vom Mittelwert abweicht
# p-value: wenn unter 0.05 lässt das auf einen Zusammenhang zwischen X und Y schließen


import statsmodels.api as sm

est = sm.OLS(y, X)
stat = est.fit()
print(stat.summary())

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


                                 OLS Regression Results                                
Dep. Variable:           Nackenumfang   R-squared (uncentered):                   0.999
Model:                            OLS   Adj. R-squared (uncentered):              0.999
Method:                 Least Squares   F-statistic:                          1.838e+04
Date:                Tue, 10 Nov 2020   Prob (F-statistic):                        0.00
Time:                        09:48:20   Log-Likelihood:                         -394.40
No. Observations:                 252   AIC:                                      816.8
Df Residuals:                     238   BIC:                                      866.2
Df Model:                          14                                                  
Covariance Type:            nonrobust                                                  
                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------

In [7]:
### Feature Selection mit Variance Threshhold
# Daten mit niedriger bis keiner Varianz werden nichts zum Model beitragen, also entfernen

from sklearn.feature_selection import VarianceThreshold


selector = VarianceThreshold(threshold=0.8)
selector.fit_transform(X)
selector.get_support()

# 0 (Dichte) wurde aussortiert
print(selector.get_support(indices=True))

X = filter_selection(X, selector)

# Dichte wurde entfernt

X.head()

[ 1  2  3  4  5  6  7  8  9 10 11 12 13]


,Koerperfettanteil,Alter,Gewicht_lbs,Groesse_inch,Brustumfang,Bauchumfang,Hueftumfang,Obschenkelumfang,Knieumfang,Fussgelenkumfang,Oberarmumfang,Unterarmumfang,Handgelenkumfang
0,12.3,23,154.25,67.75,93.1,85.2,94.5,59.0,37.3,21.9,32.0,27.4,17.1
1,6.1,22,173.25,72.25,93.6,83.0,98.7,58.7,37.3,23.4,30.5,28.9,18.2
2,25.3,22,154.00,66.25,95.8,87.9,99.2,59.6,38.9,24.0,28.8,25.2,16.6
3,10.4,26,184.75,72.25,101.8,86.4,101.2,60.1,37.3,22.8,32.4,29.4,18.2
4,28.7,24,184.25,71.25,97.3,100.0,101.9,63.2,42.2,24.0,32.2,27.7,17.7


In [8]:
### Feature Selection mit Univariate Feature Selection (K-Best)
# Features werden bepunktet und nur die k-besten werden zurückgegeben
# Es wird eine Funktion festgelegt, anhand derer die Bepunktung stattfindet

# Für f_regression werden F-Tests durchgeführt. Es werden alle Features
# geprüft auf den F-Test, der beste wird gewählt (M_1). Es wird dann ein weiteres
# Feature in Kombination mit M_1 aus den übrigen Features getestet. Dies wird
# solange wiederholt, bis die k-besten Features ausgewählt wurden.

# Der F-Test ist ein statistisches Verfahren, bei dem unter der Nullhypothese
# der Zusammenhang zwischen statistischen Populationen geprüft wird.

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

selector = SelectKBest(f_regression, k=5)
X_freg   = selector.fit_transform(X, y)

X_freg   = filter_selection(X, selector)

X_freg

,Gewicht_lbs,Brustumfang,Bauchumfang,Hueftumfang,Handgelenkumfang
0,154.25,93.1,85.2,94.5,17.1
1,173.25,93.6,83.0,98.7,18.2
2,154.00,95.8,87.9,99.2,16.6
3,184.75,101.8,86.4,101.2,18.2
4,184.25,97.3,100.0,101.9,17.7
...,...,...,...,...,...
247,134.25,89.2,83.6,88.8,18.5
248,201.00,108.5,105.0,104.5,20.1
249,186.75,111.1,111.5,101.7,18.0
250,190.75,108.3,101.3,97.8,19.8


In [9]:
### Feature Selection mit Univariate Feature Selection (K-Best)
# Features werden bepunktet und nur die k-besten werden zurückgegeben
# Es wird eine Funktion festgelegt, anhand derer die Bepunktung stattfindet

# Mutual Information quantifiziert die Menge an Information die eine Variable
# einer statistischen Population über eine andere ausdrückt. Das Konzept ist
# stark mit dem der Entropie verbunden.

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression

selector = SelectKBest(mutual_info_regression, k=5)
X_mut    = selector.fit_transform(X, y)

X_mut    = filter_selection(X, selector)

X_mut

,Gewicht_lbs,Brustumfang,Hueftumfang,Oberarmumfang,Unterarmumfang
0,154.25,93.1,94.5,32.0,27.4
1,173.25,93.6,98.7,30.5,28.9
2,154.00,95.8,99.2,28.8,25.2
3,184.75,101.8,101.2,32.4,29.4
4,184.25,97.3,101.9,32.2,27.7
...,...,...,...,...,...
247,134.25,89.2,88.8,25.6,25.7
248,201.00,108.5,104.5,35.2,28.6
249,186.75,111.1,101.7,31.3,27.2
250,190.75,108.3,97.8,30.5,29.4


In [11]:
### Feature Selection Model basierend
# RandomForestRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
import numpy as np

regr = RandomForestRegressor(n_estimators=100)
regr = regr.fit(X, y)
prob = -regr.feature_importances_

# Abstimmung ausgeben
print(prob)

# Indizes ermitteln
idx = np.argpartition(prob, -4)[-4:]

print(idx)

# Reduziertes Dataframe ausgeben
X_for = data_frame.iloc[:, idx]

print(X_for)

[-0.01730937 -0.01563635 -0.42860141 -0.00989114 -0.13155285 -0.08319753
 -0.02258944 -0.02960141 -0.02012168 -0.03479305 -0.03336419 -0.07200505
 -0.10133652]
[8 0 3 1]
     Hueftumfang  Dichte  Gewicht_lbs  Koerperfettanteil
0           94.5  1.0708       154.25               12.3
1           98.7  1.0853       173.25                6.1
2           99.2  1.0414       154.00               25.3
3          101.2  1.0751       184.75               10.4
4          101.9  1.0340       184.25               28.7
..           ...     ...          ...                ...
247         88.8  1.0736       134.25               11.0
248        104.5  1.0236       201.00               33.6
249        101.7  1.0328       186.75               29.3
250         97.8  1.0399       190.75               26.0
251        107.1  1.0271       207.50               31.9

[252 rows x 4 columns]
